## Add acne to face


In [89]:
import cv2
import numpy as np
import os

def process_images(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')): 
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path)
            if image is not None:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                faces = detect_faces(image)
                processed_image = simulate_acne_on_faces(image, faces, num_acne=50, min_size=3, max_size=12)
                output_image_path = os.path.join(output_folder, filename)
                cv2.imwrite(output_image_path, cv2.cvtColor(processed_image, cv2.COLOR_RGB2BGR))

# 偵測臉部區域
def detect_faces(image):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    return faces

# 偵測眼睛位置
def detect_eyes(image, face):
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    (x, y, w, h) = face
    roi_gray = image[y:y+h, x:x+w]
    eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=10)
    return [(ex+x, ey+y, ew, eh) for (ex, ey, ew, eh) in eyes]

# 在臉部區域隨機撒模擬痘痘
def simulate_acne_on_faces(image, faces, num_acne=5, min_size=5, max_size=20):
    output = image.copy()
    for (x, y, w, h) in faces:
        face_center_region = (x + int(w * 0.15), y + int(h * 0.2), int(w * 0.7), int(h * 0.6))
        fx, fy, fw, fh = face_center_region
        eyes = detect_eyes(image, (x, y, w, h))
        for _ in range(num_acne):
            valid = False
            while not valid:
                center_x = np.random.randint(fx, fx + fw)
                center_y = np.random.randint(fy, fy + fh)
                size = np.random.randint(min_size, max_size)
                valid = True
                # 確認痘痘不要長在眼睛上
                for (ex, ey, ew, eh) in eyes:
                    if (ex < center_x < ex + ew) and (ey < center_y < ey + eh):
                        valid = False
                        break
                if not (fx <= center_x - size and center_x + size <= fx + fw and fy <= center_y - size and center_y + size <= fy + fh):
                    valid = False
            random_value = np.random.rand()
            if random_value < 0.33:
                color = (139, 62, 47) 
            elif random_value < 0.66:
                color = (205, 112, 84)
            else:
                color = (238, 106, 80)
            cv2.circle(output, (center_x, center_y), size, color, -1)
    return output

input_folder = 'All_image'
output_folder = 'New_image'
process_images(input_folder, output_folder)


## Cut and resized images

In [ ]:
import cv2
import os
import numpy as np

source_folder = './noacne_white_background_2444_1718_541/'
target_folder = './noacne_white_background_1024_1024_541/'

if not os.path.exists(target_folder):
    os.makedirs(target_folder)

for filename in os.listdir(source_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(source_folder, filename)
        image = cv2.imread(image_path)
        if image is None:
            continue
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 設置裁剪的座標位置
        x, y, w, h = 363, 0, 1718, 1718

        # 裁剪圖片
        cropped_image = image_rgb[y:y+h, x:x+w]

        # 調整圖片大小
        resized_image = cv2.resize(cropped_image, (1024, 1024))

        target_path = os.path.join(target_folder, filename)
        cv2.imwrite(target_path, cv2.cvtColor(resized_image, cv2.COLOR_RGB2BGR))

print("處理完成，所有圖片已保存至", target_folder)


## Resized images

In [ ]:
import cv2
import os
import numpy as np

source_folder = './Inpainting_result/Inpainting_result/'
target_folder = './Inpainting__new/'

if not os.path.exists(target_folder):
    os.makedirs(target_folder)

for filename in os.listdir(source_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(source_folder, filename)
        image = cv2.imread(image_path)

        if image is None:
            continue

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # 調整圖片大小
        resized_image = cv2.resize(image_rgb, (512, 512))

        target_path = os.path.join(target_folder, filename)
        cv2.imwrite(target_path, cv2.cvtColor(resized_image, cv2.COLOR_RGB2BGR))

print("處理完成，所有圖片已保存至", target_folder)


## Rename the Images

In [ ]:
import os

def rename_images_sequentially(folder_path):
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']  # 支持的圖片格式
    files = [file for file in os.listdir(folder_path) if any(file.lower().endswith(ext) for ext in image_extensions)]
    files.sort()

    # 重新命名
    for index, filename in enumerate(files, start=1):

        # 得到檔案的副檔名
        ext = os.path.splitext(filename)[1]

        # 生成新的文件名
        new_filename = f"{index}{ext}"
        
        old_file = os.path.join(folder_path, filename)
        new_file = os.path.join(folder_path, new_filename)

        # 重新命名文件
        os.rename(old_file, new_file)
        print(f"Renamed '{filename}' to '{new_filename}'")

folder_path = 'noacne_white_background_1024_1024_541'
rename_images_sequentially(folder_path)


## Merge two images

In [ ]:
import os
from PIL import Image

def merge_images(image_path1, image_path2, output_path):
    img1 = Image.open(image_path1)
    img2 = Image.open(image_path2)
    if img1.height != img2.height:
        img2 = img2.resize((int(img2.width * img1.height / img2.height), img1.height))
    total_width = img1.width + img2.width
    new_img = Image.new('RGB', (total_width, img1.height))
    new_img.paste(img1, (0, 0))
    new_img.paste(img2, (img1.width, 0))
    new_img.save(output_path)

def process_folders(folder_a, folder_b, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    filenames_a = set(os.listdir(folder_a))
    filenames_b = set(os.listdir(folder_b))

    common_filenames = filenames_a.intersection(filenames_b)  # 找出兩個資料夾中都有的文件名

    for filename in common_filenames:
        file_path_a = os.path.join(folder_a, filename)
        file_path_b = os.path.join(folder_b, filename)
        output_path = os.path.join(output_folder, filename)
        merge_images(file_path_a, file_path_b, output_path)
        print(f"Merged {filename} saved to {output_path}")

folder_a = 'noacne_white_background_1024_1024_541'
folder_b = 'acne_white_background_1024_1024_541'
output_folder = 'aline_noacne2acne_white_background_1024_1024_541'
process_folders(folder_a, folder_b, output_folder)


## Change the background color (random)

In [ ]:
import os
import random
from PIL import Image, ImageColor

def get_low_saturation_color():
    # 隨機生成一個低飽和度的顏色
    hue = random.randint(0, 360)  # 色調範圍從0到360
    saturation = random.randint(10, 30)  # 低飽和度範圍從10%到30%
    value = random.randint(50, 90)  # 亮度範圍從50%到90%
    return ImageColor.getrgb(f"hsv({hue}, {saturation}%, {value}%)")

def change_background_color(image_path, output_path, threshold=245):
    img = Image.open(image_path)
    img = img.convert("RGBA")
    new_bg_color = get_low_saturation_color()
    data = img.getdata()
    new_data = []
    for item in data:
        if item[0] > threshold and item[1] > threshold and item[2] > threshold:
            new_data.append(new_bg_color + (item[3],))
        else:
            new_data.append(item)
    img.putdata(new_data)
    img = img.convert("RGB")  # Convert back to RGB before saving
    img.save(output_path)

def process_folder(input_folder, output_folder, threshold=245):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        file_path = os.path.join(input_folder, filename)
        if file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            output_path = os.path.join(output_folder, filename)
            change_background_color(file_path, output_path, threshold)
            print(f"Processed {filename} with a unique low saturation background.")

input_folder = 'aline_noacne2acne_white_background_1024_1024_541'
output_folder = 'aline_noacne2acne_color_background_1024_1024_541'
process_folder(input_folder, output_folder)
